In [1]:
import cv2
import torch
import numpy as np
from torchvision import transforms
from utils.datasets import letterbox
from utils.torch_utils import select_device
from models.experimental import attempt_load
from utils.general import non_max_suppression_kpt, strip_optimizer
from utils.plots import colors, plot_one_box_kpt

In [2]:
strip_optimizer("0", "yolov7-w6-pose.pt")
weights="yolov7-w6-pose.pt"
device = select_device("0")

model = attempt_load(weights, map_location=device) 
_ = model.eval()
names = model.module.names if hasattr(model, "module") else model.names 

Optimizer stripped from yolov7-w6-pose.pt, 161.1MB
Fusing layers... 


In [160]:
src = cv2.imread("pose1.jpg")
dst = src.copy()
dst = cv2.resize(dst, (640*src.shape[1]//src.shape[0], 640))
dst2 = cv2.resize(dst, (640*src.shape[1]//src.shape[0], 640))
img = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
img = letterbox(img, (640), stride=64, auto=True)[0]
img = transforms.ToTensor()(img)
img = torch.tensor(np.array([img.numpy()]))
img = img.to(device)
img = img.float()
with torch.no_grad():
    output_data, _ = model(img)
output_data = non_max_suppression_kpt(
                output_data,
                0.25,
                0.65,
                nc=model.yaml["nc"],
                nkpt=model.yaml["nkpt"],
                kpt_label=True)

for det in output_data[0]:
    if det[4] < 0.9:
        continue
    for i in range(len(det)):
        if i in [6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54]:
            dst = cv2.circle(dst, (int(det[i]), int(det[i+1])), 3, (0, 255, 0), -1, cv2.LINE_AA)
            dst = cv2.putText(dst, "{:.2f}".format(det[i+2].item()), (int(det[i]+5), int(det[i+1]-4)), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.imshow("dst", dst)
        # if cv2.waitKey() == 27:
        #     break
        # else:
        #     continue
# for pose in output_data:
    # for det_index, (*xyxy, conf, cls) in enumerate(reversed(pose[:,:6])):
    #     c = int(cls) 
    #     kpts = pose[det_index, 6:]
    #     label = f"{names[c]} {conf:.2f}"
    #     plot_one_box_kpt(xyxy, dst2, label=label, color=colors(c, True), 
    #                 line_thickness=3, kpt_label=True, kpts=kpts, steps=3, 
    #                 orig_shape=dst2.shape[:2])
    # break
cv2.imshow("dst", dst)
# cv2.imshow("dst2", dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [149]:
output_data[0][0][11].item()

0.9883378148078918